In [99]:
import pandas as pd

In [100]:
fact19 = pd.read_csv('C:/Users/HP/Documents/DATA/[MAZSQL22] SQL for Data Analytics/PayTM/fact19.csv')
fact20 = pd.read_csv('C:/Users/HP/Documents/DATA/[MAZSQL22] SQL for Data Analytics/PayTM/fact20.csv')
dim_payment_channel = pd.read_csv('C:/Users/HP/Documents/DATA/[MAZSQL22] SQL for Data Analytics/PayTM/dim_payment_channel.csv')
dim_platform = pd.read_csv('C:/Users/HP/Documents/DATA/[MAZSQL22] SQL for Data Analytics/PayTM/dim_platform.csv')
dim_scenario = pd.read_csv('C:/Users/HP/Documents/DATA/[MAZSQL22] SQL for Data Analytics/PayTM/dim_scenario.csv')
dim_status = pd.read_csv('C:/Users/HP/Documents/DATA/[MAZSQL22] SQL for Data Analytics/PayTM/dim_status.csv')

In [101]:
dim_payment_channel.head(3)

,payment_channel_id,payment_method
0,11,local card
1,12,bank account
2,13,balance


In [102]:
dim_platform.head(3)

,platform_id,payment_platform
0,P1,others
1,P2,android
2,P3,ios


In [103]:
dim_status.head(3)

,status_id,status_description
0,1,Success
1,-2,security error
2,-3,receiver system error


In [104]:
dim_scenario.head(3)

,scenario_id,transaction_type,category,sub_category
0,S1_10,Internet bank transfers,shopping,NaN
1,S1_100,Internet bank transfers,not payment,NaN
2,S1_101,Internet bank transfers,NaN,NaN


In [131]:
df = pd.concat([fact19,fact20])

In [106]:
df.head(3)

,transaction_id,customer_id,scenario_id,payment_channel_id,promotion_id,platform_id,status_id,original_price,discount_value,charged_amount,transaction_time
0,101,3203,S2_1,12,0,P2,1,100000,0,100000,2019-01-01 00:01:00.0000000
1,102,2324,S2_1,12,0,P2,1,300000,0,300000,2019-01-01 00:25:00.0000000
2,103,18641,S2_1,12,0,P1,1,100000,0,100000,2019-01-01 00:22:00.0000000


In [107]:
from datetime import datetime

In [132]:
# chuyển định dạng ngày 
df['transaction_time'] = df['transaction_time'].apply(lambda x: datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S.%f0'))

In [133]:
#  tạp cột year
df['year'] = df['transaction_time'].apply(lambda x: x.year)

#  tạo cột month
df['month'] = df['transaction_time'].apply(lambda x: x.month)


1. Retrieve a report that includes the following information: customer_id, transaction_id, scenario_id, transaction_type, category, payment_method. 
These transactions must meet the following conditions: 
Were created from Jan to June 2019, 
Had category type is shopping,
Were paid by Bank account


In [110]:
df_1 = df.loc[(df['month'].isin([2,7])) & (df['year']==2019)] \
    .merge(dim_scenario, how='left', on='scenario_id') \
    .merge(dim_payment_channel,  how= 'left', on = 'payment_channel_id') \
    [['customer_id', 'transaction_id', 'scenario_id', 'transaction_type','category', 'payment_method']]

df_1 = df_1.loc[(df_1['payment_method']== 'bank account') & (df_1['category'] == 'shopping')]
df_1

,customer_id,transaction_id,scenario_id,transaction_type,category,payment_method
128,6883,22036,S1_13,top-up account,shopping,bank account
199,8264,22107,S1_10,Internet bank transfers,shopping,bank account
246,19972,22154,S1_13,top-up account,shopping,bank account
247,21618,22155,S1_10,Internet bank transfers,shopping,bank account
440,13423,22349,S1_13,top-up account,shopping,bank account
...,...,...,...,...,...,...
51331,2542,179316,S1_13,top-up account,shopping,bank account
51910,5230,179895,S1_13,top-up account,shopping,bank account
51926,27349,179911,S1_150,NaN,shopping,bank account
51967,17871,179952,S1_13,top-up account,shopping,bank account


2. In the fact_transaction_2019 data table, each successful transaction corresponds to a different category. 
Calculate the number of transactions of each category to make reporting easier.

In [111]:
df_2 = fact19.merge(dim_status, how= 'left', on = 'status_id').merge(dim_scenario, how = 'left', on='scenario_id')
df_2 = df_2.loc[df_2['status_description'] == 'Success'].groupby('category')['transaction_id'].agg(total = 'count').reset_index()
df_2

,category,total
0,billing,78406
1,fnb,2452
2,marketplace,6541
3,not payment,12511
4,shopping,52911
5,telco,985
6,traveling,183406


3. Please indicate the total number of transactions of each transaction_type, provided: successful transaction, transaction time in the first 3 months of 2019
Calculate the ratio of the number of transactions of each type to the total number of transactions in 3 months.

In [112]:
trans_type = df.merge(dim_status, on = 'status_id').merge(dim_scenario, on ='scenario_id')
trans_type = trans_type.loc[(trans_type['year']==2019) \
    & (trans_type['month'].isin([1,4])) \
    & (trans_type['status_description']=='Success')]\
    .groupby('transaction_type')['transaction_id'].agg(num_trans ='count').reset_index()

trans_type['ratio'] = trans_type['num_trans'].apply(lambda x: '{:.2%}'.format(x/trans_type['num_trans'].sum()))
trans_type

,transaction_type,num_trans,ratio
0,Internet bank transfers,13257,44.10%
1,creadit card billing,1697,5.64%
2,payments,804,2.67%
3,top-up account,7624,25.36%
4,withdraw,6681,22.22%


4. Retrieve a more detailed report with following information: transaction type, category, number of transactions and proportion of each category in the total of that transaction type. 
These transactions must meet the following conditions: 
Were created in 2019 
Were paid successful

In [113]:
df_4 = fact19.merge(dim_status, on ='status_id').merge(dim_scenario, on='scenario_id')
trans_type = df_4.loc[df_4['status_description'] =='Success'].groupby('transaction_type')['transaction_id'].agg(num_trans ='count').reset_index()
df_4 = df_4.loc[df_4['status_description'] =='Success'] \
    .groupby(['transaction_type','category'])['transaction_id'].agg(total ='count').reset_index() \
    .merge(trans_type, on='transaction_type')

df_4['pct'] = (df_4['total']/df_4['num_trans']).apply(lambda x: '{:.2%}'.format(x))
df_4

,transaction_type,category,total,num_trans,pct
0,Internet bank transfers,billing,132,116923,0.11%
1,Internet bank transfers,fnb,42,116923,0.04%
2,Internet bank transfers,marketplace,253,116923,0.22%
3,Internet bank transfers,not payment,18,116923,0.02%
4,Internet bank transfers,shopping,4296,116923,3.67%
5,Internet bank transfers,telco,7,116923,0.01%
6,Internet bank transfers,traveling,112172,116923,95.94%
7,creadit card billing,billing,5299,19327,27.42%
8,creadit card billing,fnb,1,19327,0.01%
9,creadit card billing,not payment,12320,19327,63.75%


5.  Hãy tính xem số lượng giao dịch bị nghi ngờ gian lận của nhóm khách hàng nhận được ưu đãi là bao nhiêu

In [114]:
df_5 = df.loc[df['promotion_id'] != 0].merge(dim_status, on ='status_id')

num_trans = df_5.groupby('customer_id')['customer_id'].agg(num_trans = 'count')

fraud_trans = df_5.loc[df_5['status_description'].str.contains('fraud')]\
    .groupby('customer_id')['customer_id'].agg(num_fraud ='count').reset_index()\
    .merge(num_trans, on = 'customer_id')

fraud_trans['pct'] = (fraud_trans['num_fraud']/fraud_trans['num_trans']).apply(lambda x: '{:.2%}'.format(x))                                                                   

fraud_trans

,customer_id,num_fraud,num_trans,pct
0,109,2,140,1.43%
1,125,2,26,7.69%
2,146,2,112,1.79%
3,278,1,182,0.55%
4,344,1,20,5.00%
...,...,...,...,...
588,54171,1,28,3.57%
589,54886,1,9,11.11%
590,55663,1,3,33.33%
591,56058,1,44,2.27%


6.1  Phân loại khách hàng giao dịch thành công  thành 3 nhóm: 
Tổng số tiền giao dịch trên 5.000.000 là “New Customer”
Tổng số tiền giao dịch trên 10.000.000 là “Potential Customer”
Tổng số tiền giao dịch trên 50.000.000 là “Loyal Customer”

In [115]:
df_6 = df.merge(dim_status, on ='status_id')
df_6 = df_6.loc[df_6['status_description']=='Success'].groupby('customer_id')['charged_amount'].agg(amount='sum').reset_index()

def amount(x):
    if x> 50000000:
        return 'Loyal Customer'
    elif x > 10000000:
        return 'Potential Customer'
    elif x > 5000000:
        return 'New Customer'

df_6['type'] = df_6['amount'].apply(lambda x: amount(x))
df_6

,customer_id,amount,type
0,101,2869935,None
1,102,584961206,Loyal Customer
2,103,26562060,Potential Customer
3,104,390820550,Loyal Customer
4,105,4933900,None
...,...,...,...
62386,75994,800000,None
62387,75998,5000,None
62388,75999,3240000,None
62389,76000,19550,None


6.2. đếm type_customer và phần trăm

In [116]:
type_cus = df_6.groupby('type')['customer_id'].agg(total='count')
type_cus['pct'] = type_cus['total'].apply(lambda x: '{:.2%}'.format(x/df_6['customer_id'].count()))
type_cus

,total,pct
type,,
Loyal Customer,3874,6.21%
New Customer,3294,5.28%
Potential Customer,4937,7.91%


-- 7. Find out the TOP 3 months with the most failed transactions of each year (using window function)


In [117]:
df_7 = df.merge(dim_status, on ='status_id')
df_7 = df_7.loc[df_7['status_description'].str.contains('failed') == False].groupby(['year','month'])['transaction_id'].agg(num_trans= 'count').reset_index()
df_7['rank'] = df_7.groupby('year')['num_trans'].rank(ascending = False)
df_7 = df_7[df_7['rank'] <4][['year','month','num_trans']]
df_7

,year,month,num_trans
9,2019,10,45255
10,2019,11,45901
11,2019,12,52087
21,2020,10,86397
22,2020,11,108969
23,2020,12,125845


-- 8. Calculate the average distance between successful payments per customer in Telecom group 2019.

In [118]:
df_8 = fact19.merge(dim_scenario, on='scenario_id').merge(dim_status,on='status_id') #.sort_values('customer_id', ascending = True)
df_8 = df_8.loc[(df_8['category']=='telco') & (df_8['status_description']=='Success')][['transaction_id','customer_id','transaction_time']]

df_8['next_date'] = df_8.sort_values('transaction_time',ascending = True).groupby('customer_id')['transaction_time'].shift(-1)
# df_8['next_date'] = df_8['transaction_time'].shift(-1)
df_8['gap_days'] = (pd.to_datetime(df_8['next_date']) - pd.to_datetime(df_8['transaction_time'])).dt.days

df_8 = df_8.groupby('customer_id')['gap_days'].agg(avg_gap ='mean').reset_index()

df_8

,customer_id,avg_gap
0,102,5.533333
1,104,12.833333
2,106,6.500000
3,108,7.700000
4,109,0.000000
...,...,...
323,38558,0.000000
324,39710,NaN
325,40110,NaN
326,41179,NaN


9. You know that there are many sub-categories of the Billing group. After reviewing the above result, you should break down the trend into each sub-categories.

In [119]:
df_9 = df.merge(dim_scenario,on='scenario_id')
df_9 = df_9.loc[df_9['category']== 'billing']
df_9 = df_9.groupby(['year','month','sub_category'])['transaction_id'].agg(num_trans = 'count').reset_index()
df_9

,year,month,sub_category,num_trans
0,2019,1,credit card,4
1,2019,1,electricity,303
2,2019,1,insurance,4865
3,2019,1,internet,266
4,2019,1,tv,57
...,...,...,...,...
139,2020,12,electricity,343
140,2020,12,insurance,20175
141,2020,12,internet,2711
142,2020,12,tv,258


9.2. Then modify the result as the following table: Only select the sub-categories belong to list (Electricity, Internet and Water)

In [120]:
df_9 = df_9.loc[(df_9['sub_category']=='electricity') | (df_9['sub_category']=='internet') | (df_9['sub_category']=='water')]
df_9 = df_9.pivot_table('num_trans',['year','month'],'sub_category')
df_9

sub_category  electricity  internet  water
year month                                
2019 1              303.0     266.0  139.0
     2              120.0     147.0  199.0
     3              180.0     281.0   40.0
     4              628.0     343.0   26.0
     5              714.0     373.0    5.0
     6              762.0     403.0    5.0
     7              657.0     483.0   18.0
     8              951.0     571.0   36.0
     9              688.0     858.0   16.0
     10             865.0    1025.0    7.0
     11             586.0     908.0   14.0
     12             403.0    1114.0  285.0
2020 1              150.0    1330.0  239.0
     2              212.0    1039.0   14.0
     3               48.0    1071.0    3.0
     4               12.0     880.0    2.0
     5               29.0    1177.0   16.0
     6               31.0    1164.0   29.0
     7               45.0    1449.0   28.0
     8               27.0    1521.0  119.0
     9               91.0    1640.0  168.0
     10             248.0    2179.0  245.0
     11             189.0    2363.0  345.0
     12             343.0    2711.0  300.0

-- 10. Hãy tính trung bình cộng số lượng KH thành công đó mỗi 4 tuần (trung bình cộng của 4 tuần gần nhất)

In [121]:
df['week'] = (df['transaction_time']).dt.isocalendar().week

df_10 = df.loc[df['status_id']==1]
df_10 = df_10.groupby(['year','week'])['customer_id'].agg(num_trans='nunique').reset_index()
df_10['avg_last_4week'] = df_10['num_trans'].rolling(window=4).mean()
df_10

,year,week,num_trans,avg_last_4week
0,2019,1,3530,NaN
1,2019,2,2082,NaN
2,2019,3,2065,NaN
3,2019,4,2267,2486.00
4,2019,5,2380,2198.50
...,...,...,...,...
100,2020,49,9595,10177.00
101,2020,50,12258,10365.25
102,2020,51,10803,10677.00
103,2020,52,11467,11030.75


11. You want to evaluate the quality of user acquisition category Billing in Jan 2019 by the
retention metric. First, you need to know how many users are retained in each subsequent month
from the first month (Jan 2019) they pay the successful transaction (only get data of 2019).
Let’s calculate retention = number of retained customers / total users of the first month.

In [134]:
cus_Jan19 = df[(df['year']==2019) & (df['month']==1) & (df['status_id']==1)]['customer_id'].unique()
df_11 = df[(df['year']==2019) & (df['status_id']==1) & (df['customer_id'].isin(cus_Jan19))] \
    .groupby(['year','month'])['customer_id'] \
    .agg(retention='nunique').reset_index()


df_11['pct'] = (df_11['retention']/df_11['retention'].max()).apply(lambda x: '{:.2%}'.format(x))

df_11

,year,month,retention,pct
0,2019,1,5640,100.00%
1,2019,2,2939,52.11%
2,2019,3,2782,49.33%
3,2019,4,2591,45.94%
4,2019,5,2490,44.15%
5,2019,6,2360,41.84%
6,2019,7,2309,40.94%
7,2019,8,2318,41.10%
8,2019,9,2249,39.88%
9,2019,10,2243,39.77%


12. Cohorts Derived from the Time Series Itself
Task A: Expand your previous query to calculate retention for multi attributes from the acquisition
month (first month) (from Jan to December).


In [135]:
df_12 = df[(df['year']==2019) & (df['status_id']==1)]

df_firstmonth = df_12.groupby('customer_id')['month'].agg(first_month='min').reset_index()

df_12 = df_12.merge(df_firstmonth,on='customer_id')

df_12['subsequent_month'] = df_12['month'] - df_12['first_month']
df_12 = df_12.groupby(['first_month','subsequent_month'])['customer_id'].agg(retention = 'nunique').reset_index()
df_12['original'] = df_12.groupby('first_month')['retention'].transform('max')
df_12['pct'] = (df_12['retention']/df_12['original']).apply(lambda x: '{:.2%}'.format(x))
df_12

,first_month,subsequent_month,retention,original,pct
0,1,0,5640,5640,100.00%
1,1,1,2939,5640,52.11%
2,1,2,2782,5640,49.33%
3,1,3,2591,5640,45.94%
4,1,4,2490,5640,44.15%
...,...,...,...,...,...
73,10,1,659,1910,34.50%
74,10,2,594,1910,31.10%
75,11,0,1238,1238,100.00%
76,11,1,529,1238,42.73%


In [142]:
#  cách khác
df_cach2 = (
    df[(df['year']==2019) & (df['status_id']==1)]
    .assign(first_month = lambda x: x.groupby('customer_id')['month'].transform('min'))
    .assign(subsequent_month = lambda x: x['month'] - x['first_month'])
    .groupby(['first_month','subsequent_month'])['customer_id']
    .agg(retention = 'nunique')
    .reset_index()
    .assign(original = lambda x: x.groupby('first_month')['retention'].transform('max'))
    .assign(pct = lambda x: (x['retention']/x['original']).apply(lambda y: '{:.2%}'.format(y)))
)
df_cach2
    

,first_month,subsequent_month,retention,original,pct
0,1,0,5640,5640,100.00%
1,1,1,2939,5640,52.11%
2,1,2,2782,5640,49.33%
3,1,3,2591,5640,45.94%
4,1,4,2490,5640,44.15%
...,...,...,...,...,...
73,10,1,659,1910,34.50%
74,10,2,594,1910,31.10%
75,11,0,1238,1238,100.00%
76,11,1,529,1238,42.73%
